In [36]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import json

In [ ]:
# Cleaning empty columns out of game data
data = pd.read_csv("2025GameData.csv")

data = data[data['Rk'].astype(str).str.isnumeric()]
data = data.rename(columns={'Opp':'Opponent'})
data = data.rename(columns={'Opp.1':'Opp'})


data.to_csv("2025GameData2.csv", index=False)

In [ ]:
# Converting Game Data into Advanced Stats

# Define columns for cumulative stats
columns = ['Name', 'Game', 'Opponent', 'Site', 'Outcome', 'Date', 'ORtg', 'DRtg', '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'eFG%', 
            'TOV%', 'ORB%', 'FTr', 'oeFG%', 'oTOV%', 'oDRB%', 'oFTr', 'Pace', 'o3PAr']


year = "2025"

# Read data from CSV
data = pd.read_csv(str(year) + "GameData2.csv")

# Initialize/Reset dictionary to store totals data
team_stats = defaultdict(lambda: {
    'Games': 0, 'Points': 0, 'oPoints': 0, 'FGA': 0, 'FGM': 0, '3PA': 0, '3PM': 0,
    'FTA': 0, 'FTM': 0, 'ORB': 0, 'TRB': 0, 'AST': 0, 'STL': 0, 'BLK': 0, 'TOV': 0,
    'Poss': 0, 'oFGA': 0, 'oFGM': 0, 'o3PA': 0, 'o3PM': 0, 'oFTA': 0, 'oFTM': 0,
    'oORB': 0, 'oTRB': 0, 'oTOV': 0, 'oPoss': 0
})

# Initialize/Reset cumulative_stats DataFrame
cumulative_stats = pd.DataFrame(columns=columns)

# Loop through each row of data to aggregate stats by team
for i in tqdm(range(len(data)), desc=year, unit="game"):
    team_name = data.iloc[i]['Team']
    
    # Convert each item in the current row to a float if it looks like a number
    data.iloc[i] = data.iloc[i].apply(lambda x: float(x) if isinstance(x, str) and x.replace('.', '', 1).isdigit() else x)

    # Count the number of successfully converted floats
    float_count = sum(isinstance(value, float) for value in data.iloc[i])

    # Count the number of empty or NaN values in the row
    empty_count = data.iloc[i].isnull().sum()

    # Skip the row if it has no convertible floats or if it has more than 1 empty value
    if float_count == 0 or empty_count > 2:
        continue

    # Update team data in the dictionary
    team_stats[team_name]['Games'] += 1
    team_stats[team_name]['Points'] += data.iloc[i]['Tm']
    team_stats[team_name]['FGA'] += data.iloc[i]['FGA']
    team_stats[team_name]['FGM'] += data.iloc[i]['FG']
    team_stats[team_name]['3PA'] += data.iloc[i]['3PA']
    team_stats[team_name]['3PM'] += data.iloc[i]['3P']
    team_stats[team_name]['FTA'] += data.iloc[i]['FTA']
    team_stats[team_name]['FTM'] += data.iloc[i]['FT']
    team_stats[team_name]['ORB'] += data.iloc[i]['ORB']
    team_stats[team_name]['TRB'] += data.iloc[i]['TRB']
    team_stats[team_name]['AST'] += data.iloc[i]['AST']
    team_stats[team_name]['STL'] += data.iloc[i]['STL']
    team_stats[team_name]['BLK'] += data.iloc[i]['BLK']
    team_stats[team_name]['TOV'] += data.iloc[i]['TOV']
    team_stats[team_name]['Poss'] += data.iloc[i]['FGA'] - data.iloc[i]['ORB'] + data.iloc[i]['TOV'] + 0.475 * data.iloc[i]['FTA']
    team_stats[team_name]['oPoints'] += data.iloc[i]['Opp']
    team_stats[team_name]['oFGA'] += data.iloc[i]['oFGA']
    team_stats[team_name]['oFGM'] += data.iloc[i]['oFG']
    team_stats[team_name]['o3PA'] += data.iloc[i]['o3PA']
    team_stats[team_name]['o3PM'] += data.iloc[i]['o3P']
    team_stats[team_name]['oFTA'] += data.iloc[i]['oFTA']
    team_stats[team_name]['oFTM'] += data.iloc[i]['oFT']
    team_stats[team_name]['oORB'] += data.iloc[i]['oORB']
    team_stats[team_name]['oTRB'] += data.iloc[i]['oTRB']
    team_stats[team_name]['oTOV'] += data.iloc[i]['oTOV']
    team_stats[team_name]['oPoss'] += data.iloc[i]['oFGA'] - data.iloc[i]['oORB'] + data.iloc[i]['oTOV'] + 0.475 * data.iloc[i]['oFTA']

    # Convert only the current team's stats to a DataFrame
    current_team_stats = pd.DataFrame([{'Name' : team_name,
                                        'Site' : data.iloc[i]['Site'],
                                        'Date' : data.iloc[i]['Date'],
                                        'Opponent' : data.iloc[i]['Opponent'],
                                        'Outcome' : int(data.iloc[i]['Tm'] > data.iloc[i]['Opp']),
                                        'Game' : team_stats[team_name]['Games'],
                                        'ORtg' : 100 * (team_stats[team_name]['Points'] / team_stats[team_name]['Poss']),
                                        'DRtg' : 100 * (team_stats[team_name]['oPoints'] / team_stats[team_name]['oPoss']),
                                        '3PAr' : team_stats[team_name]['3PA'] / team_stats[team_name]['FGA'],
                                        'TS%' : team_stats[team_name]['Points'] / (2 * (team_stats[team_name]['FGA'] + 0.475 * team_stats[team_name]['FTA'])),
                                        'TRB%' : team_stats[team_name]['TRB'] / (team_stats[team_name]['TRB'] + team_stats[team_name]['oTRB']),
                                        'AST%' : team_stats[team_name]['AST'] / team_stats[team_name]['FGM'],
                                        'STL%' : team_stats[team_name]['STL'] / team_stats[team_name]['oPoss'],
                                        'BLK%' : team_stats[team_name]['BLK'] / (team_stats[team_name]['oFGA'] - team_stats[team_name]['o3PA']),
                                        'eFG%' : (team_stats[team_name]['FGM'] + 0.5 * team_stats[team_name]['3PM']) / team_stats[team_name]['FGA'],
                                        'TOV%' : team_stats[team_name]['TOV'] / team_stats[team_name]['Poss'],
                                        'ORB%' : team_stats[team_name]['ORB'] / (team_stats[team_name]['ORB'] + team_stats[team_name]['oTRB'] - team_stats[team_name]['oORB']),
                                        'FTr' : team_stats[team_name]['FTA'] / team_stats[team_name]['FGA'],
                                        'oeFG%' : (team_stats[team_name]['oFGM'] + 0.5 * team_stats[team_name]['o3PM']) / team_stats[team_name]['oFGA'],
                                        'oTOV%' : team_stats[team_name]['TOV'] / team_stats[team_name]['oPoss'],
                                        'oDRB%' : (team_stats[team_name]['oTRB'] - team_stats[team_name]['oORB']) / (team_stats[team_name]['oTRB'] - team_stats[team_name]['oORB'] + team_stats[team_name]['ORB']),
                                        'oFTr' : team_stats[team_name]['oFTA'] / team_stats[team_name]['oFGA'],
                                        'Pace' : (team_stats[team_name]['Poss'] + team_stats[team_name]['oPoss']),
                                        'o3PAr' : team_stats[team_name]['o3PA'] / team_stats[team_name]['oFGA']}])

    cumulative_stats = pd.concat([cumulative_stats, current_team_stats], ignore_index=True)

# Sort by team name and then by games played within each team
cumulative_stats.sort_values(by=['Name', 'Game'], inplace=True)

cumulative_stats.to_csv('AdvWeekData' + year + '.csv', index=False, header=True)


2025:   0%|          | 0/11857 [00:00<?, ?game/s]C:\Users\dschr\AppData\Local\Temp\ipykernel_18008\131960578.py:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  cumulative_stats = pd.concat([cumulative_stats, current_team_stats], ignore_index=True)
2025: 100%|██████████| 11857/11857 [01:30<00:00, 130.74game/s]


In [ ]:
data2 = pd.read_csv("AdvWeek_CSVs/AdvWeekData2025.csv")

# Get the index of the row with the max "Game" value for each "Team"
latest_game_idx = data2.groupby('Name')['Game'].idxmax()

# Use that index to pull the full rows
latest_games = data2.loc[latest_game_idx].reset_index(drop=True)

latest_games.to_csv("FinalStats2025.csv")

for i in range(11):
    year = str(i + 2014)

    data2 = pd.read_csv("AdvWeek_CSVs/AdvWeekData" + year + ".csv")

    # Get the index of the row with the max "Game" value for each "Team"
    latest_game_idx = data2.groupby('Name')['Game'].idxmax()

    # Use that index to pull the full rows
    latest_games = data2.loc[latest_game_idx].reset_index(drop=True)

    latest_games.to_csv("FinalStats" + year + ".csv")


In [ ]:
# Creating input data for model

# Define columns for input data
columns = ['Team 1', 'Team 2', 'Date', 'Outcome', '1-ORtg', '1-DRtg', '1-3PAr', '1-TS%', '1-TRB%',
           '1-AST%', '1-STL%', '1-BLK%', '1-eFG%', '1-TOV%', '1-ORB%', '1-FTr', '1-oeFG%', '1-oTOV%', '1-oDRB%',
           '1-oFTr', '1-Pace', '1-o3PAr', '2-ORtg', '2-DRtg', '2-3PAr', '2-TS%', '2-TRB%', '2-AST%', '2-STL%',
           '2-BLK%', '2-eFG%', '2-TOV%', '2-ORB%', '2-FTr', '2-oeFG%', '2-oTOV%', '2-oDRB%', '2-oFTr', '2-Pace', '2-o3PAr']

# Load in Tournament dates
with open('dates.json', 'r') as f:
    dates = json.load(f)

for i in range(12):
    year = 2014 + i
    if year == 2020:
        continue

    # Load in Tournament teams and game data
    week_data = pd.read_csv("AdvWeek_CSVs/AdvWeekData" + str(year) + ".csv")
    with open("Tournament_Teams/Tournament_Teams_" + str(year) + ".txt", 'r') as f:
        teams = [line.strip() for line in f if line.strip()]

    # Filtering to only pre-Tournament games
    first_round_date = pd.to_datetime(dates[str(year)]["1st Round"])
    week_data['Date'] = pd.to_datetime(week_data['Date'])
    pre_tourney_games = week_data[week_data['Date'] < first_round_date]

    # Grabbing the last game for each tournament team
    last_games = pre_tourney_games[pre_tourney_games['Name'].isin(teams)]
    last_games = last_games.sort_values(by='Date').groupby('Name').tail(1)





    

